<a href="https://colab.research.google.com/github/chrislin1009/shishow/blob/main/Final_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***期末報告 : LINE BOT - chatgpt 你的心理諮商師***

## 背景與動機

本專案旨在將 OpenAI 的 ChatGPT API 整合至 LINE Bot 平台，使用者無需額外下載或學習新的工具，即可透過熟悉的 LINE 聊天介面與 AI 對話，獲得即時回應與互動體驗。此外，透過角色扮演功能，此系統可以透過心理諮商師的口吻，提供心情的抒發及引導，協助使用者走出心情鬱悶亦或是解決心中所產生的疑難雜症。

## 目標

-建立一個整合 OpenAI ChatGPT API 的 LINE 聊天機器人。

-設定 AI 的對話風格為心理諮商師，透過提問引導使用者深入思考。

-提供自然語言輸入與回應的互動介面，讓使用者可在 LINE 中輕鬆與 AI 對話。

-實現穩定運作的後端系統，支援上下文記憶與角色持續性，提升對話流暢度與真實感。

## 系統設計與架構

使 chatgpt 扮演成諮商師的角色 :


*   具有同理心，能夠站在對方立場理解其感受與處境，而非只是同情或指導。
*   彈性與創造力，面對不同個案時能彈性調整策略，並創造適合當事人的介入方式。
*   批判性思考與分析能力，能從言語、行為與背景中分析出心理問題的可能來源，做出合理判斷。

*   倫理判斷與決策能力，面對道德兩難時能依據專業倫理做出合適選擇。





## 系統功能分析

| 功能項目 | 說明 |
|----------|------|
| 使用者訊息接收 | 使用者透過 LINE 傳送訊息，Bot 接收並處理 |
| Gemini API 連接 | 將使用者訊息轉交至 Google Gemini 並取得 AI 回覆 |
| 回應傳送 | Bot 將 Gemini 的回應以文字方式回傳給使用者 |
| 基本指令處理 | 回覆「幫助」等內建說明指令，提供使用說明 |
| 錯誤處理 | 在 API 異常或超時時提供提示訊息 |

## 使用的技術

| 技術項目 | 說明 |
|----------|------|
| 程式語言 | Python |
| 框架 | Flask（建立 webhook） |
| 部署平台 | Versel |
| API 使用 | Google Gemini API（文字生成），LINE Messaging API（對話） |
| 資料格式 | JSON 處理傳輸與接收資料 |

## 成果展示

In [ ]:
!pip install flask
!pip install pyngrok
!pip install flask_ngrok
!pip install line-bot-sdk

from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def ngrok_start():
  ngrok.set_auth_token(userdata.get('NGROK'))
  ngrok.connect(5000)
  run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get("LINE_CHANNEL_ACCESS_TOKEN"))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'




### ChatGPT 配置
import openai
from openai import OpenAI

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

def ask_chatgpt(user_input):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": (
                "你的身分是一位心理諮商師，面對不同個案時能彈性調整策略，並創造適合當事人的介入方式。"
                "具有批判性思考與分析能力，能從言語、行為與背景中分析出心理問題的可能來源，做出合理判斷。"
                "具有同理心，能夠站在對方立場理解其感受與處境，而非只是同情或指導。"
            )},
            {"role": "user", "content": user_input}
        ]
    )
    return response.choices[0].message.content


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        user_msg = event.message.text
        response = ask_chatgpt(user_msg)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=response)]
            )
        )

ngrok_start()  # for colab

if __name__ == "__main__":
    app.run()


## 未來發展與改進

### 改進建議  
- 加入「上下文記憶」功能，讓 AI 理解前後對話關聯   
- 強化錯誤處理與用戶提示，提升使用者體驗

### 未來規劃  
- 整合圖像生成功能（如 Gemini 支援圖片輸出）  
- 支援語音輸入與 TTS 回覆  
- 能切換成不同角色或是性別進行回答